# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [44]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ciudad lazaro cardenas,17.9583,-102.2000,28.62,76,71,2.17,MX,1723129292
1,1,port-aux-francais,-49.3500,70.2167,1.31,71,99,12.04,TF,1723129293
2,2,grytviken,-54.2811,-36.5092,-7.52,82,100,5.62,GS,1723129294
3,3,wailua homesteads,22.0669,-159.3780,21.53,81,28,3.58,US,1723129295
4,4,kirakira,-10.4544,161.9205,25.43,88,100,4.28,SB,1723129296


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    color='City',
    alpha=0.5,
    tiles='OSM',
    frame_height=510,
    frame_width=800,
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [47]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]
# Drop any rows with null values
ideal_weather_df_clean = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df_clean


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
113,113,vila franca do campo,37.7167,-25.4333,26.30,57,0,2.22,PT,1723129421
124,124,guerrero negro,27.9769,-114.0611,23.57,62,0,4.09,MX,1723129433
161,161,shimoda,34.6667,138.9500,26.41,87,0,1.58,JP,1723129478
175,175,newman,37.3138,-121.0208,22.80,59,0,1.79,US,1723129494
199,199,mahanoro,-19.9000,48.8000,21.83,76,0,2.70,MG,1723129520
204,204,constantia,44.1833,28.6500,25.41,56,0,3.27,RO,1723129526
209,209,fenoarivo atsinanana,-17.3667,49.4167,21.63,74,0,2.66,MG,1723129531
305,305,sandy,40.5720,-111.8606,26.68,30,0,1.79,US,1723129641
308,308,arkadelphia,34.1209,-93.0538,25.87,74,0,0.00,US,1723129645
339,339,korla,41.7597,86.1469,26.90,32,0,4.30,CN,1723129679


### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df_clean[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
113,vila franca do campo,PT,37.7167,-25.4333,57,
124,guerrero negro,MX,27.9769,-114.0611,62,
161,shimoda,JP,34.6667,138.9500,87,
175,newman,US,37.3138,-121.0208,59,
199,mahanoro,MG,-19.9000,48.8000,76,
204,constantia,RO,44.1833,28.6500,56,
209,fenoarivo atsinanana,MG,-17.3667,49.4167,74,
305,sandy,US,40.5720,-111.8606,30,
308,arkadelphia,US,34.1209,-93.0538,74,
339,korla,CN,41.7597,86.1469,32,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories": "accommodation.hotel",
    "filter": "",
    "bias": "",
    "limit": 1,
    "apiKey": geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
guerrero negro - nearest hotel: Plaza sal paraiso
shimoda - nearest hotel: ホテルウラガ
newman - nearest hotel: No hotel found
mahanoro - nearest hotel: Vanilla Bungalow
constantia - nearest hotel: Hotel Dali
fenoarivo atsinanana - nearest hotel: Bungalows Faniry
sandy - nearest hotel: Best Western Plus Cottontree Inn
arkadelphia - nearest hotel: Captain Henderson House
korla - nearest hotel: Silver Star Hotel
colonia - nearest hotel: Wasserturm Hotel Cologne
seymour - nearest hotel: Quality Inn & Suites
cole harbour - nearest hotel: No hotel found
commerce - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
113,vila franca do campo,PT,37.7167,-25.4333,57,Vinha d'Areia Beach Hotel
124,guerrero negro,MX,27.9769,-114.0611,62,Plaza sal paraiso
161,shimoda,JP,34.6667,138.9500,87,ホテルウラガ
175,newman,US,37.3138,-121.0208,59,No hotel found
199,mahanoro,MG,-19.9000,48.8000,76,Vanilla Bungalow
204,constantia,RO,44.1833,28.6500,56,Hotel Dali
209,fenoarivo atsinanana,MG,-17.3667,49.4167,74,Bungalows Faniry
305,sandy,US,40.5720,-111.8606,30,Best Western Plus Cottontree Inn
308,arkadelphia,US,34.1209,-93.0538,74,Captain Henderson House
339,korla,CN,41.7597,86.1469,32,Silver Star Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [52]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    color='City',
    alpha=0.5,
    tiles='OSM',
    frame_height=500,
    frame_width=800,
    hover_cols=['City', 'Country','Humidity', 'Hotel Name']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)